<a href="https://colab.research.google.com/github/RaviKcse08/datascience_projs/blob/main/DC_Intro_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from pprint import pprint

In [28]:
sd = pd.read_csv('bank_marketing.csv')
#print(sd.shape)
sd.head()
dframe = sd.dropna()
dframe.head()
df = dframe.rename(columns={"y": "label"})
df = df.drop(['job','marital','education','default','housing','loan','contact','putcome','month'], axis=1)
df.head()

,age,balance,day,duration,,pdays,previous,label
0,51,6530,5,91,1,-1,0,no
1,27,7459,9,407,2,-1,0,no
2,59,13308,13,131,1,-1,0,no
3,48,7373,14,126,1,-1,0,no
4,51,7180,14,927,1,-1,0,yes


In [29]:
train_df, test_df = train_test_split(df, test_size=20)
train_df.shape
#data[:5]

(35114, 8)

In [30]:
#helper function to check uncertainity
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [31]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]    
    return classification

TypeError: ignored

In [35]:
def get_potential_splits(data):
    
    potential_splits = {}
    n_records, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [36]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

In [37]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [38]:
def decision_tree_algorithm(df, counter=0):
    
    # data preparations
    if counter == 0:
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if check_purity(data):
        classification = classify_data(data)
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        question = "{} <= {}".format(split_column, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter)
        no_answer = decision_tree_algorithm(data_above, counter)
        
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
        
        return sub_tree

In [39]:
tree = decision_tree_algorithm(train_df)
pprint(tree)

{'3 <= 383.5': [{'5 <= 19.5': [{'3 <= 130.5': [{'3 <= 77.5': [{'4 <= 3.5': [{'0 <= 72.5': [{'3 <= 62.5': [{'3 <= 39.5': [{'4 <= 1.5': [{'2 <= 30.5': [{'3 <= 38.5': [{'0 <= 27.5': [{'2 <= 9.5': [{'2 <= 8.5': ['no',
                                                                                                                                                                                                                {'3 <= 23.5': [{'3 <= 19.0': ['no',
                                                                                                                                                                                                                                              'yes']},
                                                                                                                                                                                                                               'no']}]},
                                                              